In [1]:
import pandas as pd
import numpy as np

class SME():
    
    def __init__(self):
        self.asked = 0
        self.df = self.get()
        
    def get(self):
        X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')
        y_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_y.csv')

        X_train['Left'] = y_train
        return X_train
    
    def ask(self, valuedict):
        self.asked += 1
        #  print(self.asked)
        
        if self.asked > 500000:
            raise Exception("Sorry, you have asked enough")

        arr = []
        for prop in valuedict:
            val = valuedict[prop]

            if val is None: 
                continue

            val = f"'{val}'" if isinstance(val,str) else val
            # print(prop, val)
            arr.append(f'{prop} == {val}')

            query = ' and '.join(arr)
        result = self.df.query(query)

        if len(result) == 0:
            raise Exception("I don't know")
        
        return (int)(result['Left'].mean())

In [2]:
sme = SME()

In [3]:
sme.df.columns

Index(['RecordId', 'Gender', 'Distance', 'YearsWorked', 'TrainingHours',
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'DepartmentCode', 'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Left'],
      dtype='object')

In [4]:
X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')

In [5]:

sme.df['SelfReview'] = sme.df['SelfReview'].fillna(sme.df['SelfReview'].median())
sme.df['SupervisorReview'] = sme.df['SupervisorReview'].fillna(sme.df['SupervisorReview'].median())
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].map(lambda x: x.replace('K', '000') if isinstance(x, str) else x)
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].astype(int)

sme.df['Raise'] = (sme.df['Salary'] / sme.df['PreviousSalary']) - 1
sme.df['ReviewDiff'] = sme.df['SelfReview'] - sme.df['SupervisorReview']
sme.df['ReviewOverRaise'] = sme.df['SelfReview'] / (sme.df['Raise'] +.0001)

sme.df['LongDist'] = np.where(sme.df['Distance'].isin(['~20miles', '>30miles']), 1, 0)

sme.df['HighStress'] = np.where(sme.df['StressLevel'] > 3.5, 1, 0)

num = ['HighStress', 'LongDist','Gender', 'SupervisorReview','SelfReview', 'Raise', 'Salary','ReviewOverRaise', 'PreviousSalary'] #, 'LongDist', ReviewOverRaise','ReviewDiff', 'StressLevel'
cat = ['DepartmentCode']#'Distance',
#num = ['Raise', 'StressLevel']

"""
num = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', 
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']
"""

"\nnum = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', \n       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',\n       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',\n       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',\n       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',\n       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',\n       'PreviousSalary', 'Salary', 'SelfReview',\n       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']\n"

In [6]:
sme.df[num].corr()>.1

,HighStress,LongDist,Gender,SupervisorReview,SelfReview,Raise,Salary,ReviewOverRaise,PreviousSalary
HighStress,True,False,False,False,False,False,False,False,False
LongDist,False,True,False,False,False,False,False,False,False
Gender,False,False,True,False,False,False,False,False,False
SupervisorReview,False,False,False,True,False,False,False,False,False
SelfReview,False,False,False,False,True,False,False,False,False
Raise,False,False,False,False,False,True,False,False,False
Salary,False,False,False,False,False,False,True,False,True
ReviewOverRaise,False,False,False,False,False,False,False,True,False
PreviousSalary,False,False,False,False,False,False,True,False,True


# Code for Feature Engineering

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X[['Salary', 'PreviousSalary']] = X[['Salary', 'PreviousSalary']].map(lambda x: x.replace('K', '000') if isinstance(x, str) else x)
        X[['Salary', 'PreviousSalary']] = X[['Salary', 'PreviousSalary']].astype(int)
        X['Raise'] = (X['Salary'] / X['PreviousSalary']) - 1
        X['ReviewDiff'] = X['SelfReview'] - X['SupervisorReview']
        X['ReviewOverRaise'] = X['SelfReview'] / (X['Raise'] +.0001)
        return X

# Establish the preprocessing for numeric and categorical features.

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans

n_clusters = 500

featurepreprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num)#,
        #('cat', OneHotEncoder(), cat)
    ])
preprocessor = Pipeline(steps=[
    #('featureengineering', FeatureEngineering()),
    ('features', featurepreprocessor),
    ('pca', PCA(n_components=.9)),#.9
    ('kmeans', KMeans(n_clusters=n_clusters, random_state=42, n_init='auto'))
    ])
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto'), num)#+cat)
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto')

In [11]:
X_dist = preprocessor.fit_transform(sme.df[num+cat])

In [12]:
preprocessor

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['HighStress', 'LongDist',
                                                   'Gender', 'SupervisorReview',
                                                   'SelfReview', 'Raise',
                                                   'Salary', 'ReviewOverRaise',
                                                   'PreviousSalary'])])),
                ('pca', PCA(n_components=0.9)),
                ('kmeans',
                 KMeans(n_clusters=500, n_init='auto', random_state=42))])

In [13]:
kmeans = KMeans(n_clusters = n_clusters, n_init='auto')
pre = featurepreprocessor.fit_transform(sme.df[num+cat])

In [14]:
X_dist = kmeans.fit_transform(pre)
X_dist

array([[3.88900685, 4.33915901, 3.44198542, ..., 3.60420852, 3.19105408,
        2.8652429 ],
       [3.77400286, 2.67538029, 2.80724733, ..., 3.71565166, 4.13210852,
        4.11027283],
       [3.4848853 , 4.41402426, 2.75686476, ..., 3.34832875, 5.04950794,
        4.03051275],
       ...,
       [4.40355945, 4.49697223, 3.70310195, ..., 1.8670727 , 4.84992318,
        5.41646518],
       [5.12308396, 2.06113089, 1.80106865, ..., 4.8322161 , 5.72972063,
        4.48900559],
       [2.47145272, 5.27722795, 4.41121128, ..., 4.65346816, 2.28339671,
        2.54676681]])

In [15]:
X_dist.shape

(500000, 500)

In [16]:
#How many points are in each cluster?
pd.Series(kmeans.labels_).value_counts()

187    3464
369    3401
171    2627
102    2572
39     2470
       ... 
318     222
402     212
423     211
92      203
250     202
Name: count, Length: 500, dtype: int64

In [17]:
import numpy as np
#What are the reporesentative points? (e.i the ones closest to each centroid)
representative_idx = np.argmin(X_dist, axis=0)
representative_idx.shape

(500,)

In [18]:
#What is the index number of each representative point?
representative_idx

array([ 19861,   8018,   1064,  11098,   1351,  58439,   2456,  16751,
         2913,   2147,   6509,   1718,   1186,   2523,    846,    635,
         5696,   2551,  10140, 118514,  19991,  11039,   2521,    419,
         2106,    668,  36182,  17695,    660,  54681,   9459,   6787,
         3668,   9555,  35093,   1846,     85,  18880,   3241,    885,
        20272,   2935,   3163,   3607,  30872,   6566,   1783, 148083,
        24867,  12097,   1669,   2597,    469,  45808,    459,  33233,
        14224,   4641,   1876,  21183,   5026,   1360,  78569,  27396,
         8369,    516,     68,   2941,    475,   7377,  28697,   5991,
        11402,   3730,  10766,  12603,   1628,   1555,    163,   9627,
         3934,   6138,  14097,  14776,  15626,   9309,    452,  31964,
         1642,    275,  68326,   8103, 200196,    624,  28398,   7771,
         4558,   6517,    416,   2995,   2958,    367,    482,  76756,
        21997,   6972,   4998,   5091,   3121,  57980,  20044,    292,
      

In [19]:
#What do the reporesentative points look like?
X_representative = sme.df.iloc[representative_idx]
X_representative

,RecordId,Gender,Distance,YearsWorked,TrainingHours,WorkLifeBalance,NumOfProjects,JobInvolvement,TeamSize,MentorshipReceived,...,PreviousSalary,Salary,SelfReview,SupervisorReview,Left,Raise,ReviewDiff,ReviewOverRaise,LongDist,HighStress
19861,19961,1,~15miles,5,31,3,11,2,10,1,...,81000,83000,5.0,3.0,1,0.024691,2.0,201.683183,0,1
8018,8118,0,~15miles,4,5,1,9,5,5,1,...,51000,53000,4.0,5.0,0,0.039216,-1.0,101.740562,0,0
1064,1164,0,~10miles,5,5,1,1,2,13,1,...,65000,66000,5.0,4.0,0,0.015385,1.0,322.901143,0,0
11098,11198,0,~20miles,9,79,3,7,2,14,0,...,81000,83000,4.0,4.0,0,0.024691,0.0,161.346546,1,0
1351,1451,0,~20miles,5,8,1,13,3,8,1,...,54000,56000,5.0,4.0,0,0.037037,1.0,134.636481,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3504,3604,0,~10miles,7,24,5,18,3,12,1,...,85000,88000,4.0,5.0,0,0.035294,-1.0,113.013129,0,0
17283,17383,0,~15miles,7,65,5,8,1,12,0,...,57000,60000,5.0,2.0,1,0.052632,3.0,94.819842,0,0
24016,24116,0,>30miles,7,28,1,6,3,10,0,...,67000,70000,4.0,2.0,1,0.044776,2.0,89.134267,1,0
697,797,1,~20miles,9,64,3,11,4,12,1,...,84000,87000,4.0,4.0,0,0.035714,0.0,111.687276,1,1


In [20]:
#Look up the leave/stay value of hte 500 representative points
y_representative= []
for i in representative_idx:
    y_representative.append(sme.ask({'RecordId': i+100})) #All record Id's are 100 more than the index value

In [21]:
#Propagate the label value to each point in the cluster.
y_train_propagated = np.empty(len(sme.df), dtype=np.int32)
for i in range(n_clusters):
    y_train_propagated[kmeans.labels_ == i] = y_representative[i]

In [22]:
#Assign the labels and leave/stay values to the DF
sme.df['Labels'] = kmeans.labels_
sme.df['y_pred'] = y_train_propagated

In [58]:
sme.df

,RecordId,Gender,Distance,YearsWorked,TrainingHours,WorkLifeBalance,NumOfProjects,JobInvolvement,TeamSize,MentorshipReceived,...,SupervisorReview,Left,Raise,ReviewDiff,ReviewOverRaise,LongDist,HighStress,Labels,y_pred,ClusteringOff
0,100,1,>30miles,7,91,4,14,3,8,1,...,4.0,0,0.013889,0.0,285.941223,1,0,413,0,0
1,101,0,~10miles,10,52,5,9,2,10,0,...,4.0,0,0.032258,0.0,123.616788,0,1,179,0,0
2,102,0,~10miles,10,80,3,13,3,16,0,...,2.0,1,0.013699,2.0,289.883848,0,0,372,0,1
3,103,0,~15miles,5,6,3,15,5,16,0,...,5.0,0,0.015873,0.0,313.027924,0,0,142,1,1
4,104,1,~10miles,10,3,2,13,3,18,0,...,4.0,0,0.066667,0.0,59.910135,0,0,437,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,500095,1,>30miles,9,80,1,10,3,14,0,...,5.0,0,0.032787,-1.0,121.629031,1,1,104,0,0
499996,500096,0,~20miles,7,0,3,2,5,4,1,...,5.0,0,0.033333,0.0,149.551346,1,1,204,0,0
499997,500097,0,~20miles,10,8,3,13,3,13,1,...,2.0,0,0.046154,3.0,108.099119,1,0,109,1,1
499998,500098,0,~15miles,10,51,3,7,3,5,0,...,5.0,1,0.018868,0.0,263.602905,0,0,54,0,1


In [23]:
sme.df['ClusteringOff']= abs(sme.df['y_pred'] - sme.df['Left']) #Which points are incorrectly labeled / is the clustering off for?

In [24]:
#What Percent incorrect is each cluster?
percent_df = sme.df.groupby('Labels')['ClusteringOff'].sum()/sme.df.groupby('Labels')['ClusteringOff'].count()
percent_df
#percent_df[percent_df>.5].count()

Labels
0      0.267773
1      0.156425
2      0.391376
3      0.833573
4      0.593985
         ...   
495    0.004277
496    0.521008
497    0.810047
498    0.024902
499    0.018709
Name: ClusteringOff, Length: 500, dtype: float64

In [25]:
#What is the total accuracy of the model?
1- sme.df['ClusteringOff'].sum() / sme.df['ClusteringOff'].count()

0.759252

In [26]:
#What percentage of "Leaving" predictions did our model get wrong?
sme.df[sme.df['Left']==1]['ClusteringOff'].sum() / sme.df[sme.df['Left']==1]['ClusteringOff'].count()

0.3532175927208649

In [27]:
sme.df[sme.df['Left']==1]['ClusteringOff'].sum() / sme.df[sme.df['Left']==1]['ClusteringOff'].count()

0.3532175927208649

In [28]:
#How many "Stays" should be "leaves?
sme.df[sme.df['Left']==1]['ClusteringOff'].sum()

63742

In [29]:
#How many "leaves" Should be "stays"?
sme.df[sme.df['Left']==0]['ClusteringOff'].sum()

56632

# Cleaning Pipeline to set up for ML